In [1]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import os
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
import getpass
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
import sys
sys.tracebacklimit = 0
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import PyPDFLoader
import pandas as pd
import numpy as np

c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")
OPENAI_MODEL_GPT4 = os.environ.get("OPENAI_MODEL_GPT4")
OPENAI_DEPLOYMENT_GPT4 = os.environ.get("OPENAI_DEPLOYMENT_GPT4")

In [3]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)
llm_gpt4 = AzureChatOpenAI(model_name=OPENAI_MODEL_GPT4, azure_deployment=OPENAI_DEPLOYMENT_GPT4,temperature=0)

In [4]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
    )
    return result

In [5]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False,
    )
    return result

In [5]:
questions = [
    "Analyze the following quote from Dr. Volumnia Gaul from 'Ballad of Songbirds and Snakes': 'There is a point to everything or nothing at all, depending on your worldview.'",
    "Analyze the following quote from 'Ballad of Songbirds and Snakes': 'He'd loved the unfamiliar sense of safety that their defeat had brought. The security that could only come with power. The ability to control things. Yes, that was what he'd loved best of all.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'Snow lands on top.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': '...at eighteen, the heir to the once-great house of Snow had nothing to live on but his wits.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'Bombs and blood. That was how the rebels had killed his mother. He wondered if they had killed Lucy Gray's, too. 'Just like her pearly white bones.' She seemed to have no love for District 12, always separating herself from it, saying she was, what was it . . . Covey?'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'All right, so he'd dropped the handkerchief with Lucy Gray's scent - the one from the outside pocket of his book bag - into the snake tank. He'd done it so they would not bite her as they had Clemensia. So they would not kill her. Because he cared about her. Because he cared about her? Or because he wanted to win the Hunger Games so that he could secure the Plinth Prize. If it was the latter, he had cheated to win, and that was that.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'The Hunger Games are an unnatural, vicious punishment. How could a good person like you be expected to go along with them?'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'If I'm helping to kill people in the districts, how is it any better than helping to kill them in the Hunger Games?'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'You don't think I've invested all this time in you to hand you off to those imbeciles in the districts, do you?'"
]


ground_truths = [
    ["Throughout The Ballad, Dr. Gaul presents Coryo and the other students with items of political philosophy, often in order to explain her own views of power and control. Here, her contrast between a 'point' and 'nothing' plays into a few of her larger ideas. Authoritarian though she is, Dr. Gaul sees the power of the Capitol as serving a useful point in rescuing Panem from chaos. On another level, the situation that provokes this comment - Coryo and Clemensia observing Dr. Gaul's multicolored snakes and wondering if there is a 'point' to the coloration - reveals that the idea of purpose extends well beyond vast political constructs in the novel. Dr. Gaul's rainbow snakes serve a direct point in making the Hunger Games a stark spectacle, and they serve a narrative point in connecting Lucy with one of her symbolic animals in the arena. It is clear what worldview Dr. Gaul would endorse."],
    ["Here, Coryo is completing an assignment for Dr. Gaul on the rather twisted topic of what he liked about the war, and he is proving just how compatible Dr. Gaul's authoritarian worldview is with his own. Of course, Coryo's reasons for disliking the Districts are not purely sociopathic. The warfare caused his family real suffering and gave him the impression that, despite his apparent intelligence and his potential for decency, he was, for a time, fundamentally helpless. Still, is it reassuring 'security' or maniacal 'power' that really appeals to Coriolanus? On the basis of the later events of his life, his desire for control and power becomes twisted beyond understandable self-preservation and into open villainy."],
    ["This is something of a Snow family motto for Tigris and Coryo: the 'saying that had gotten them through the war' (Ballad, 9), as the two cousins recollect early in the novel. Though loaded with reassurance and a pleasing sense of inevitability, this mantra varies in tone, context, and implication as The Ballad of Songbirds and Snakes progresses. 'Snow lands on top' appears early on, when Tigris is contemplating her family's poverty and Snow is eager to excel as a mentor. These are inspiring words for difficult circumstances, but by the final time the motto appears—in the Epilogue, with Coryo victorious and increasingly vicious—a Snow has in fact landed on top and has succeeded in crushing plenty of other people."],
    ["Here, we are told that Snow is only eighteen, and we are led to contemplate the differences between the wealthy dictator of the later Hunger Games trilogy and the earnest young Coryo we see here. His kindly mother and his domineering father are dead, and he doesn't have their fortune—or any fortune—to benefit him, since his family's resources were lost in the rebellion with the presumed destruction of District 13. Now, Snow must use his wits and cunning to escape his situation. Thus, the book sets up Coryo's need to restore his family's name as one of the character's central motivations. Readers of the final trilogy may know the end result of Snow's life, but the task of connecting this impoverished young man to the well-heeled, sociopathic President Snow offers plenty of intrigue."],
    ["Is the bond between Snow and Lucy mainly based on self-interest or on genuine affection? While a romance between an aristocrat from the Capitol and a wandering musician may seem to have a shaky foundation, Snow's early reflections indicate that he feels some sympathy for Lucy. Their shared experience of the war is one of loneliness and displacement. Still, as Snow's need to excel takes precedence, his connection to Lucy becomes more complicated. The idea that Lucy has 'no love for District 12,' for instance, will later be deployed by Snow in discussions and interviews meant to appeal to Capitol sponsors. Of course, construing Lucy as non-District may help her popularity and her odds of winning, but it is impossible to separate Snow's bond with Lucy from his own determination to improve his life."],
    ["The narration of The Ballad is by no means indirect about the moral dilemmas that Coryo encounters. Here, the narrative explicitly raises the question of whether he desires Lucy's safety or a personal win more, underscoring this theme with sentences that are identical but for a single unit of punctuation: 'Because he cared about her. Because he cared about her?' At this point in the novel, it is possible to read a single action as doubled-sided and Coryo's confusion as genuine. By the end, however, he will decisively choose his future over Lucy, reaching a level of self-interest that he doesn't definitively attain in this scene."],
    ["There are a few Capitol citizens who object to the Hunger Games in a principled manner, though not all of them speak out as sharply as Sejanus Plinth does. Among the mentors, Lysistrata Vickers seems to harbor real compassion for her tribute, and Coriolanus himself is troubled - at least for a time - by the fact that he kills a tribute while escaping the arena. Tigris's own objections are, from one perspective, surprising. Though related to a Capitol loyalist (her grandmother) and a future authoritarian (her cousin), she finds the games miserable and believes that Coryo does the same. Her judgment of Coriolanus may be flawed and her idea may not quite fit her family's sympathies."],
    ["In part, Sejanus's decision to become a Peacekeeper was meant to remove him from a Capitol milieu that he never found appealing. Nonetheless, this quotation reveals that Sejanus is in an impossible position. If becoming a Peacekeeper - ideally, a Peacekeeper doctor or medic - was meant as a form of escape and a route to atonement, then Sejanus's new approach has led him down the wrong path. He is forced to help an institution that executes rebels without trial and that, for the most part, is willing to let the Districts remain in disrepair - beyond providing the Capitol with resources, that is."],
    ["Here, Dr. Gaul reveals that, in her own supremely self-interested way, she holds Coriolanus in high regard. He is a fantastic investment, and his time in District 12 has proven that he can thrive even in conditions of deprivation and uncertainty despite his Capitol pedigree. Dr. Gaul's own Capitol elitism is on display in her reference to the 'imbeciles' from the Districts. How much, however, did Coryo really learn from his time in the Districts? His stint as a Peacemaker is construed as a necessary toughening in The Ballad, but the events of the later Hunger Games trilogy suggest that Coriolanus comes to underestimate those District 'imbeciles' at his own peril."]
   ]

In [6]:
columns = ["System", "Faithfulness", "Answer Relevancy", "Context Precision", "Context Recall", "Answer Similarity", "Answer Correctness"]
results_df = pd.DataFrame(columns=columns)

In [7]:
max_average = 0
def find_highest(average_score):
    global max_average
    if average_score > max_average:
        max_average = average_score
        print("This is the new best value!")

In [8]:
def dictionary(result):
    dict_result = dict(result)
    average_score = sum(dict_result.values()) / len(dict_result)
    print(f"The average score is: {average_score}")
    find_highest(average_score)
    return average_score

In [10]:
def evaluate_system(system_name, questions, answers, contexts, ground_truths):
    result = evaluation_rag(questions, answers, contexts, ground_truths)
    average = dictionary(result)
    # Create a dictionary to store the results
    system_results = {
        "System": system_name,
        "Faithfulness": result["faithfulness"],
        "Answer Relevancy": result["answer_relevancy"],
        "Context Precision": result["context_precision"],
        "Context Recall": result["context_recall"],
        "Answer Similarity": result["answer_similarity"],
        "Answer Correctness": result["answer_correctness"],
        "Average": average
    }
    df_system_results = pd.DataFrame([system_results])
    return df_system_results

In [19]:
def evaluate_LLM(system_name, questions, answers, contexts, ground_truths):
    result = evaluation_llm(questions, answers, contexts, ground_truths)
    average = dictionary(result)
    # Create a dictionary to store the results
    system_results = {
        "System": system_name,
        "Faithfulness": result["faithfulness"],
        "Answer Relevancy": result["answer_relevancy"],
        "Context Precision": np.nan,
        "Context Recall": np.nan,
        "Answer Similarity": result["answer_similarity"],
        "Answer Correctness": result["answer_correctness"],
        "Average": average
    }
    df_llm_results = pd.DataFrame([system_results])
    return df_llm_results

### General answer by LLMs

In [10]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)
llm_chain_gpt4 =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm_gpt4}
)

In [12]:
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""],[""]]

In [13]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

In [16]:
results_df = pd.read_csv(r"C:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\ballad\results\results_report.csv")

In [22]:
llm_results = evaluate_LLM("GPT-3.5", questions, answers_llm, contexts_llm, ground_truths)
results_df = pd.concat([results_df, llm_results], ignore_index=True)
print(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 36/36 [00:07<00:00,  4.68it/s]


The average score is: 0.8169907355965818
This is the new best value!
    System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0  GPT-3.5           1.0          0.741668                NaN             NaN   

   Answer Similarity  Answer Correctness   Average  
0           0.866096            0.660199  0.816991  


In [23]:
answers_llm_gpt4 = []
for query in questions:
    response = llm_chain_gpt4.invoke({"question": query})
    answers_llm_gpt4.append(response["response"].content)
llm_results_gpt4 = evaluate_LLM("GPT-4",questions, answers_llm_gpt4, contexts_llm, ground_truths)
results_df = pd.concat([results_df, llm_results_gpt4], ignore_index=True)
print(llm_results_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 36/36 [00:06<00:00,  5.50it/s]


The average score is: 0.8278482200227925
This is the new best value!
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0  GPT-4       0.90625          0.889053                NaN             NaN   

   Answer Similarity  Answer Correctness   Average  
0           0.858152            0.657938  0.827848  


In [24]:
results_df.to_csv(r"C:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\ballad\results\results_report.csv", index = False)

# RAG

### Naive RAG

In [17]:
def retrieval_chain(prompt, retriever, llm):
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    return retrieval_augmented_qa_chain

In [18]:
loader = PyPDFLoader(r"..\ballad\the_ballad_of_songbirds_and_snakes.pdf")
documents = loader.load()

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db_naive = Chroma.from_documents(chunks, embeddings_client, persist_directory = "../ballad/vectordb/naive")
db_naive.persist()
retriever_naive = db_naive.as_retriever()

In [19]:
db_naive = Chroma(persist_directory = "../ballad/vectordb-edit/naive", embedding_function=embeddings_client)
retriever_naive = db_naive.as_retriever()

In [20]:
template = """User input {question}. 
Context {context}."""
prompt = ChatPromptTemplate.from_template(template)

In [21]:
answers_naive = []
contexts_naive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_naive, llm).invoke({"question": query})
        # Access the response content
        answers_naive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_naive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_naive.append("No answer")
        context_full = retriever_naive.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_naive.append(context_content)

In [22]:
result_naive_rag = evaluate_system("Naive", questions, answers_naive, contexts_naive, ground_truths)
results_df = pd.concat([results_df, result_naive_rag], ignore_index=True)
print(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7338979625335442
This is the new best value!
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0  Naive      0.840282          0.726358             0.5125        0.764092   

   Answer Similarity  Answer Correctness   Average  
0            0.75473            0.805426  0.733898  


C:\Users\sigitalapina\AppData\Local\Temp\ipykernel_14248\3254586886.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, result_naive_rag], ignore_index=True)


### recursive splitter

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()
chunks_r = text_splitter.split_documents(documents)
db_basic = Chroma.from_documents(chunks_r, embeddings_client, persist_directory = "../ballad/vectordb/recursive_basic")
db_basic.persist()
retriever_basic = db_basic.as_retriever()

In [23]:
db_basic = Chroma(persist_directory = "../ballad/vectordb/recursive_basic", embedding_function=embeddings_client)
retriever_basic = db_basic.as_retriever()

In [24]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_basic, llm).invoke({"question": query})
        # Access the response content
        answers_recursive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_recursive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_recursive.append("No answer")
        context_full = retriever_basic.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_recursive.append(context_content)

In [25]:
result_recursive = evaluate_system("Recursive", questions, answers_naive, contexts_naive, ground_truths)
results_df = pd.concat([results_df, result_recursive], ignore_index=True)
result_recursive

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.6915253422166995


,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,Recursive,0.840282,0.720529,0.455556,0.542151,0.78868,0.801954,0.691525


### chunk sizes

In [26]:
def run_and_evaluate(name, retriever, prompt, llm, results_df):
    answers = []
    contexts_extra = []

    for query in questions:
        try:  
            response = retrieval_chain(prompt, retriever, llm).invoke({"question": query})
            # Access the response content
            answers.append(response["response"].content)
            # Access the context content
            context_content = [context.page_content for context in response["context"]]
            contexts_extra.append(context_content)  
        except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers.append("No answer")
            context_full = retriever.get_relevant_documents(query)
            context_content = [context.page_content for context in context_full]
            contexts_extra.append(context_content)

    result = evaluate_system(name, questions, answers, contexts_extra, ground_truths)
    results_df = pd.concat([results_df, result], ignore_index=True)
    return result, results_df

In [27]:
chunk_sizes = [500, 1000, 2000, 3000]
overlap_percentages = [0, 5, 10, 15, 20]

for chunk_size in chunk_sizes:
    for overlap_percentage in overlap_percentages:
        # Calculate overlap based on percentage
        chunk_overlap = int(chunk_size * overlap_percentage / 100)
        
        # Create text splitter
        # text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        
        # # Split documents
        # chunks = text_splitter.split_documents(documents)
        
        # Print number of chunks
        print(f"Number of chunks for chunk size {chunk_size}, overlap {overlap_percentage}%")
        
        # Create Chroma database
        # db = Chroma.from_documents(chunks, embeddings_client, persist_directory=f"../ballad/vectordb-edit/chunking_{chunk_size}_{overlap_percentage}")
        db = Chroma(persist_directory = "../ballad/vectordb/recursive_basic", embedding_function=embeddings_client)
        # db.persist()
        
        # Create retriever
        retriever = db.as_retriever()
        
        # Run and evaluate
        result,results_df = run_and_evaluate(f"Chunk {chunk_size}, overlap {overlap_percentage}%", retriever, prompt, llm, results_df)
        print(f"CHUNK SIZE {chunk_size}, {overlap_percentage}% overlap:")
        print(result)

Number of chunks for chunk size 500, overlap 0%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.6882996033812755
CHUNK SIZE 500, 0% overlap:
                  System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 500, overlap 0%      0.784614          0.716773           0.492593   

   Context Recall  Answer Similarity  Answer Correctness  Average  
0        0.653206           0.678285            0.804328   0.6883  
Number of chunks for chunk size 500, overlap 5%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7324911155497603
CHUNK SIZE 500, 5% overlap:
                  System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 500, overlap 5%      0.893852          0.735855           0.470833   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0         0.80184           0.678389            0.814178  0.732491  
Number of chunks for chunk size 500, overlap 10%


...

The average score is: 0.7566066604996909
This is the new best value!
CHUNK SIZE 3000, 10% overlap:
                    System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 3000, overlap 10%       0.89577          0.721733             0.6375   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0        0.800931           0.678137            0.805568  0.756607  
Number of chunks for chunk size 3000, overlap 15%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7153330543302842
CHUNK SIZE 3000, 15% overlap:
                    System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 3000, overlap 15%       0.87487          0.733177             0.5125   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0        0.575943           0.789802            0.805707  0.715333  
Number of chunks for chunk size 3000, overlap 20%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.6961263020327757
CHUNK SIZE 3000, 20% overlap:
                    System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 3000, overlap 20%      0.838542          0.723454           0.470833   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0        0.653299           0.678285            0.812345  0.696126  


chunk = 1000

In [ ]:
# # THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
# chunks_1000 = text_splitter.split_documents(documents)
# print(len(chunks_1000))
# db_1000 = Chroma.from_documents(chunks_1000, embeddings_client, persist_directory = "../ballad/vectordb/recursive_1000")
# db_1000.persist()

In [28]:
# db_1000 = Chroma(persist_directory = "../ballad/vectordb/recursive_1000", embedding_function=embeddings_client)

In [29]:
# retriever_1000 = db_1000.as_retriever()
# result_1000 = run_and_evaluate(retriever_1000, prompt, llm)
# print("CHUNK SIZE 1000")
# print(result_1000)
# avg_result_1000, dict_result_1000 = dictionary(result_1000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 1000
{'faithfulness': 0.8073, 'answer_relevancy': 0.7830, 'context_precision': 0.4722, 'context_recall': 0.7878, 'answer_similarity': 0.6856, 'answer_correctness': 0.8029}
The average score is: 0.7231345250780993


chunk = 500

In [33]:
# # THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 50)
# chunks_500 = text_splitter.split_documents(documents)
# print(len(chunks_500))
# db_500 = Chroma.from_documents(chunks_500, embeddings_client, persist_directory = "../ballad/vectordb/recursive_500")
# db_500.persist()

107


In [30]:
# db_500 = Chroma(persist_directory = "../ballad/vectordb/recursive_500", embedding_function=embeddings_client)

In [31]:
# retriever_500 = db_500.as_retriever()
# result_500 = run_and_evaluate(retriever_500, prompt, llm)
# print("CHUNK SIZE 500")
# print(result_500)
# avg_result_500, dict_result_500 = dictionary(result_500)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-202' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-203' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-204' coro=<AsyncClient.aclose() done, defined

CHUNK SIZE 500
{'faithfulness': 0.8163, 'answer_relevancy': 0.9034, 'context_precision': 0.4012, 'context_recall': 0.6481, 'answer_similarity': 0.8669, 'answer_correctness': 0.6844}
The average score is: 0.720067425291858


chunk = 2000

In [35]:
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
# chunks_2000 = text_splitter.split_documents(documents)
# print(len(chunks_2000))
# db_2000 = Chroma.from_documents(chunks_2000, embeddings_client, persist_directory = "../ballad/vectordb/recursive_2000")


37


In [32]:
# db_2000 = Chroma(persist_directory = "../ballad/vectordb/recursive_2000", embedding_function=embeddings_client)

In [33]:
# retriever_2000 = db_2000.as_retriever()
# result_2000 = run_and_evaluate(retriever_2000, prompt, llm)
# print("CHUNK SIZE 2000")
# print(result_2000)
# avg_result_2000, dict_result_2000 = dictionary(result_2000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 2000
{'faithfulness': 0.8064, 'answer_relevancy': 0.7431, 'context_precision': 0.4389, 'context_recall': 0.6734, 'answer_similarity': 0.6855, 'answer_correctness': 0.7563}
The average score is: 0.6839296969751175


chunk = 3000

In [38]:
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
# chunks_3000 = text_splitter.split_documents(documents)
# print(len(chunks_3000))
# db_3000 = Chroma.from_documents(chunks_3000, embeddings_client, persist_directory = "../ballad/vectordb/recursive_3000")

32


In [34]:
# db_3000 = Chroma(persist_directory = "../ballad/vectordb/recursive_3000", embedding_function=embeddings_client)

In [35]:
# retriever_3000 = db_3000.as_retriever()
# result_3000 = run_and_evaluate(retriever_3000, prompt, llm)
# print("CHUNK SIZE 3000")
# print(result_3000)
# avg_result_3000 ,dict_result_3000 = dictionary(result_3000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 3000
{'faithfulness': 0.8087, 'answer_relevancy': 0.7442, 'context_precision': 0.5190, 'context_recall': 0.6554, 'answer_similarity': 0.7109, 'answer_correctness': 0.8236}
The average score is: 0.7103154417058323


In [28]:
results_df

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,Naive,0.840282,0.726358,0.512500,0.764092,0.754730,0.805426,0.733898
1,Recursive,0.840282,0.720529,0.455556,0.542151,0.788680,0.801954,0.691525
2,"Chunk 500, overlap 0%",0.784614,0.716773,0.492593,0.653206,0.678285,0.804328,0.688300
3,"Chunk 500, overlap 5%",0.893852,0.735855,0.470833,0.801840,0.678389,0.814178,0.732491
4,"Chunk 500, overlap 10%",0.871056,0.744101,0.470833,0.579265,0.700295,0.698068,0.677270
5,"Chunk 500, overlap 15%",0.872510,0.732657,0.562500,0.686119,0.678690,0.805707,0.723031
6,"Chunk 500, overlap 20%",0.895490,0.736052,0.512500,0.574730,0.678314,0.808880,0.700994
7,"Chunk 1000, overlap 0%",0.872622,0.733201,0.520833,0.685805,0.678690,0.804633,0.715964
8,"Chunk 1000, overlap 5%",0.870267,0.738509,0.470833,0.801253,0.677842,0.799707,0.726402
9,"Chunk 1000, overlap 10%",0.873766,0.740747,0.512500,0.579270,0.678425,0.799853,0.697427


In [29]:
results_df.to_csv(f"../ballad/results/results_report.csv", index=False)

In [30]:
highest_average = results_df["Average"].max()
print("Highest average value:", highest_average)

Highest average value: 0.7566066604996909


### now time to look for different top-k

Note: We continue with Chunk 3000, overlap 10%

In [31]:
db_k = Chroma(persist_directory = "../ballad/vectordb-edit/chunking_3000_10", embedding_function=embeddings_client)

In [32]:
k_values = [2, 3, 5, 6, 7]

# Iterate over different k values
for k in k_values:
    # Create retriever with k value
    retriever = db_k.as_retriever(search_kwargs={"k": k})
    
    # Run and evaluate
    result,results_df = run_and_evaluate(f"Chunk size 3000, overlap 20%, K={k}", retriever, prompt, llm, results_df)
    print(f"Results for K={k}:")
    print(result)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:06<00:00,  8.06it/s]


The average score is: 0.7225782984698658
Results for K=2:
                             System  Faithfulness  Answer Relevancy  \
0  Chunk size 1000, overlap 0%, K=2          0.85          0.790993   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.444444        0.796296           0.872023            0.581713   

    Average  
0  0.722578  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:07<00:00,  7.59it/s]


The average score is: 0.7057702838409203
Results for K=3:
                             System  Faithfulness  Answer Relevancy  \
0  Chunk size 1000, overlap 0%, K=3      0.666667          0.906345   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.444444        0.694444           0.872077            0.650643   

   Average  
0  0.70577  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:07<00:00,  7.71it/s]


The average score is: 0.7161028102569849
Results for K=5:
                             System  Faithfulness  Answer Relevancy  \
0  Chunk size 1000, overlap 0%, K=5      0.835714          0.899193   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.408025        0.638889           0.875473            0.639323   

    Average  
0  0.716103  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:07<00:00,  7.69it/s]


The average score is: 0.7264987379809767
Results for K=6:
                             System  Faithfulness  Answer Relevancy  \
0  Chunk size 1000, overlap 0%, K=6      0.852083          0.882043   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.415586        0.694444           0.870412            0.644423   

    Average  
0  0.726499  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7189782838222505
Results for K=7:
                             System  Faithfulness  Answer Relevancy  \
0  Chunk size 1000, overlap 0%, K=7      0.935329          0.751635   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.554894        0.810281           0.716994            0.544736   

    Average  
0  0.718978  


In [36]:
# retriever_3 = db_1000.as_retriever(search_kwargs={"k": 3})
# result_3 = run_and_evaluate(retriever_3, prompt, llm)
# print("CHUNK SIZE 1000, K=3")
# print(result_3)
# avg_result_3, dict_result_3 = dictionary(result_3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:09<00:00,  5.72it/s]


CHUNK SIZE 1000, K=3
{'faithfulness': 0.8200, 'answer_relevancy': 0.9060, 'context_precision': 0.5370, 'context_recall': 0.7963, 'answer_similarity': 0.8722, 'answer_correctness': 0.6900}
The average score is: 0.7702549797617143


In [37]:
# retriever_5 = db_1000.as_retriever(search_kwargs={"k": 5})
# result_5 = run_and_evaluate(retriever_5, prompt, llm)
# print("CHUNK SIZE 1000, K=5")
# print(result_5)
# avg_result_5, dict_result_5 = dictionary(result_5)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:10<00:00,  5.31it/s]


CHUNK SIZE 1000, K=5
{'faithfulness': 0.8667, 'answer_relevancy': 0.8925, 'context_precision': 0.4636, 'context_recall': 0.6204, 'answer_similarity': 0.8748, 'answer_correctness': 0.6645}
The average score is: 0.7303917310800102


In [38]:
# retriever_6= db_1000.as_retriever(search_kwargs={"k": 6})
# result_6 = run_and_evaluate(retriever_6, prompt, llm)
# print("CHUNK SIZE 1000, K=5")
# print(result_6)
# avg_result_6, dict_result_6 = dictionary(result_6)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:10<00:00,  5.29it/s]


CHUNK SIZE 1000, K=5
{'faithfulness': 0.8667, 'answer_relevancy': 0.9038, 'context_precision': 0.4434, 'context_recall': 0.6481, 'answer_similarity': 0.8671, 'answer_correctness': 0.6306}
The average score is: 0.7266041952543404


In [39]:
# retriever_7= db_1000.as_retriever(search_kwargs={"k": 7})
# result_7 = run_and_evaluate(retriever_7, prompt, llm)
# print("CHUNK SIZE 1000, K=7")
# print(result_7)
# avg_result_7, dict_result_7 = dictionary(result_7)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 1000, K=7
{'faithfulness': 0.9253, 'answer_relevancy': 0.7778, 'context_precision': 0.4403, 'context_recall': 0.8073, 'answer_similarity': 0.7465, 'answer_correctness': 0.5476}
The average score is: 0.7074589251052892


In [41]:
# retriever_2 = db_1000.as_retriever(search_kwargs={"k": 2})
# result_2 = run_and_evaluate(retriever_2, prompt, llm)
# print("CHUNK SIZE 1000, K=2")
# print(result_2)
# avg_result_2, dict_result_2 = dictionary(result_2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:11<00:00,  4.87it/s]


CHUNK SIZE 1000, K=2
{'faithfulness': 0.6167, 'answer_relevancy': 0.8891, 'context_precision': 0.5000, 'context_recall': 0.7778, 'answer_similarity': 0.8708, 'answer_correctness': 0.6751}
The average score is: 0.7215756009304392


In [33]:
results_df

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,Naive,0.840282,0.726358,0.512500,0.764092,0.754730,0.805426,0.733898
1,Recursive,0.840282,0.720529,0.455556,0.542151,0.788680,0.801954,0.691525
2,"Chunk 500, overlap 0%",0.784614,0.716773,0.492593,0.653206,0.678285,0.804328,0.688300
3,"Chunk 500, overlap 5%",0.893852,0.735855,0.470833,0.801840,0.678389,0.814178,0.732491
4,"Chunk 500, overlap 10%",0.871056,0.744101,0.470833,0.579265,0.700295,0.698068,0.677270
5,"Chunk 500, overlap 15%",0.872510,0.732657,0.562500,0.686119,0.678690,0.805707,0.723031
6,"Chunk 500, overlap 20%",0.895490,0.736052,0.512500,0.574730,0.678314,0.808880,0.700994
7,"Chunk 1000, overlap 0%",0.872622,0.733201,0.520833,0.685805,0.678690,0.804633,0.715964
8,"Chunk 1000, overlap 5%",0.870267,0.738509,0.470833,0.801253,0.677842,0.799707,0.726402
9,"Chunk 1000, overlap 10%",0.873766,0.740747,0.512500,0.579270,0.678425,0.799853,0.697427


### look for different retrievers


#### parent document retriever

In [38]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap = 200)
child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents",persist_directory = "../ballad/vectordb-edit/parent-report", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)
parent_document_retriever.add_documents(documents)
result_parent, results_df = run_and_evaluate(f"Parent Retriever 1000-200", parent_document_retriever, prompt, llm, results_df)
print(result_parent)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   6%|▌         | 3/54 [00:03<00:51,  1.01s/it]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'sever

The average score is: 0.7293989066263856
                      System  Faithfulness  Answer Relevancy  \
0  Parent Retriever 1000-200      0.886641          0.627272   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.666667        0.710404           0.721377            0.764034   

    Average  
0  0.729399  


In [39]:
parent_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap = 50)
child_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_small",persist_directory = "../ballad/vectordb-edit/parent_small-report", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_small = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_small,
    parent_splitter=parent_splitter_small,
)
parent_document_retriever_small.add_documents(documents)
result_parent_small, results_df = run_and_evaluate(f"Parent Retriever 500-100", parent_document_retriever_small, prompt, llm, results_df)
print(result_parent_small)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:06<00:00,  8.17it/s]


The average score is: 0.7601980077423333
This is the new best value!
                     System  Faithfulness  Answer Relevancy  \
0  Parent Retriever 500-100      0.929167          0.899692   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.614198        0.612963           0.884171            0.620998   

    Average  
0  0.760198  


In [55]:
parent_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500, chunk_overlap = 150)
child_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_large",persist_directory = "../ballad/vectordb-edit/parent_large-report", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_large = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_large,
    parent_splitter=parent_splitter_large,
)
parent_document_retriever_large.add_documents(documents)
result_parent_large , results_df = run_and_evaluate(f"Parent Retriever 1500-200", parent_document_retriever_large, prompt, llm, results_df)
print(result_parent_large)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.6419521808112125
                      System  Faithfulness  Answer Relevancy  \
0  Parent Retriever 1500-200          0.85          0.661911   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.444444        0.555556           0.848805            0.490997   

    Average  
0  0.641952  


#### Maximum marginal relevance retrieval

In [43]:
highest = results_df.nlargest(3, "Average")
highest

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
29,Parent Retriever 1500-200,0.929167,0.897697,0.654321,0.644444,0.884171,0.635567,0.774228
28,Parent Retriever 500-100,0.929167,0.899692,0.614198,0.612963,0.884171,0.620998,0.760198
19,"Chunk 3000, overlap 10%",0.895770,0.721733,0.637500,0.800931,0.678137,0.805568,0.756607


In [44]:
replacement_map = {
    'Chunk size 3000, overlap 20%, K=2': 'Chunk size 3000, 10%, K=2',
    'Chunk size 3000, overlap 20%, K=3': 'Chunk size 3000, 10%, K=3',
    'Chunk size 3000, overlap 20%, K=5': 'Chunk size 3000, 10%, K=5',
    'Chunk size 3000, overlap 20%, K=6': 'Chunk size 3000, 10%, K=6',
    'Chunk size 3000, overlap 20%, K=7': 'Chunk size 3000, 10%, K=7'
}
results_df['System'] = results_df['System'].replace(replacement_map)

In [45]:
retriever_mmr = db_k.as_retriever(search_type="mmr",)
result_mmr, results_df = run_and_evaluate(f"MMR", retriever_mmr, prompt, llm, results_df)
print("Marginal relevance")
print(result_mmr)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7042690807089732
Marginal relevance
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0    MMR      0.865144             0.757           0.561111        0.690326   

   Answer Similarity  Answer Correctness   Average  
0           0.737306            0.614728  0.704269  


#### BM25

In [47]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
chunks_3000 = text_splitter.split_documents(documents)

In [48]:
retriever_bm25 = BM25Retriever.from_documents(chunks_3000)
result_bm25, results_df = run_and_evaluate(f"BM25", retriever_bm25, prompt, llm, results_df)
print("BM25")
print(result_bm25)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:07<00:00,  7.60it/s]


The average score is: 0.7289357373657808
BM25
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0   BM25      0.820833           0.88556           0.537037        0.666667   

   Answer Similarity  Answer Correctness   Average  
0             0.8833            0.580217  0.728936  


#### Ensambler - Hybrid

In [50]:
retriever_3000 = db_k.as_retriever()

In [51]:
ensemble_retriever_1 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3000], weights=[0.75, 0.25])
result_ensemble1, results_df = run_and_evaluate(f"Ensambler 1", ensemble_retriever_1, prompt, llm, results_df)
print("Ensambler")
print(result_ensemble1)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7340846401449582
Ensambler
        System  Faithfulness  Answer Relevancy  Context Precision  \
0  Ensambler 1      0.922353          0.737847            0.62037   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0        0.628888           0.781828            0.713222  0.734085  


In [52]:
ensemble_retriever_2 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3000], weights=[0.5, 0.5])
result_ensemble2, results_df = run_and_evaluate(f"Ensambler 2", ensemble_retriever_2, prompt, llm, results_df)
print("Ensambler")
print(result_ensemble2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7401264586357393
Ensambler
        System  Faithfulness  Answer Relevancy  Context Precision  \
0  Ensambler 2      0.886907          0.719502           0.657407   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0        0.589562           0.791583            0.795797  0.740126  


In [53]:
ensemble_retriever_3 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3000], weights=[0.25,0.75])
result_ensemble3, results_df = run_and_evaluate(f"Ensambler 3", ensemble_retriever_3, prompt, llm, results_df)
print("Ensambler")
print(result_ensemble3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:08<00:00,  6.56it/s]


The average score is: 0.7383450157672925
Ensambler
        System  Faithfulness  Answer Relevancy  Context Precision  \
0  Ensambler 3           1.0          0.868289           0.391521   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0        0.627778            0.87861            0.663873  0.738345  


#### Multi-stage - reranker

In [54]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [55]:
retriever_context = retriever_3
compressor = CohereRerank(top_n = 3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context
)

result_compression = run_and_evaluate(compression_retriever, prompt, llm)
print("Reranker")
print(result_compression)
avg_result_compression, dict_result_compression = dictionary(result_compression)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:08<00:00,  6.67it/s]


Reranker
{'faithfulness': 0.8286, 'answer_relevancy': 0.8882, 'context_precision': 0.5185, 'context_recall': 0.7407, 'answer_similarity': 0.8663, 'answer_correctness': 0.6374}
The average score is: 0.7466056282196497


In [56]:
retriever_context_more = retriever_1000
compressor = CohereRerank(top_n = 3)
compression_retriever_more = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context_more
)

result_compression_more = run_and_evaluate(compression_retriever_more, prompt, llm)
print("Reranker")
print(result_compression_more)
avg_result_compression_more, dict_result_compression_more = dictionary(result_compression_more)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

Reranker
{'faithfulness': 0.6468, 'answer_relevancy': 0.8313, 'context_precision': 0.6104, 'context_recall': 0.5648, 'answer_similarity': 0.8625, 'answer_correctness': 0.7789}
The average score is: 0.7157937819391315


#### creating context by remaking the query

In [58]:
template_context = "Generate a search query to fetch the relevant documents using the user's {question}. Craft a query that specifically targets the keywords in the question. In the answer provide only the query."
prompt_context = ChatPromptTemplate.from_template(template_context)

In [59]:
answers_final = []
contexts_final = []
llm_for_context =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm}
)
for query in questions:
    response_check = llm_for_context.invoke({"question": query})
    search_query = response_check["response"].content
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
)
    docs = parent_document_retriever_small.get_relevant_documents(search_query)
    formatted_docs = []
    for doc in docs:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final.append(response["response"].content)
            contexts_final.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final.append("No answer")
            contexts_final.append(formatted_docs)


result_search_query = evaluation_rag(questions, answers_final, contexts_final, ground_truths)
result_search_query

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:07<00:00,  7.37it/s]


{'faithfulness': 0.8125, 'answer_relevancy': 0.8838, 'context_precision': 0.4136, 'context_recall': 0.4444, 'answer_similarity': 0.8616, 'answer_correctness': 0.5940}

In [60]:
average = dictionary(result_search_query)
    # Create a dictionary to store the results
system_results = {
        "System": "Search query",
        "Faithfulness": result_search_query["faithfulness"],
        "Answer Relevancy": result_search_query["answer_relevancy"],
        "Context Precision": result_search_query["context_precision"],
        "Context Recall": result_search_query["context_recall"],
        "Answer Similarity": result_search_query["answer_similarity"],
        "Answer Correctness": result_search_query["answer_correctness"],
        "Average": average
    }
df_result_search_query = pd.DataFrame([system_results])
results_df = pd.concat([results_df, df_result_search_query], ignore_index=True)
results_df

The average score is: 0.6683153544669292


,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,Naive,0.840282,0.726358,0.512500,0.764092,0.754730,0.805426,0.733898
1,Recursive,0.840282,0.720529,0.455556,0.542151,0.788680,0.801954,0.691525
2,"Chunk 500, overlap 0%",0.784614,0.716773,0.492593,0.653206,0.678285,0.804328,0.688300
3,"Chunk 500, overlap 5%",0.893852,0.735855,0.470833,0.801840,0.678389,0.814178,0.732491
4,"Chunk 500, overlap 10%",0.871056,0.744101,0.470833,0.579265,0.700295,0.698068,0.677270
5,"Chunk 500, overlap 15%",0.872510,0.732657,0.562500,0.686119,0.678690,0.805707,0.723031
6,"Chunk 500, overlap 20%",0.895490,0.736052,0.512500,0.574730,0.678314,0.808880,0.700994
7,"Chunk 1000, overlap 0%",0.872622,0.733201,0.520833,0.685805,0.678690,0.804633,0.715964
8,"Chunk 1000, overlap 5%",0.870267,0.738509,0.470833,0.801253,0.677842,0.799707,0.726402
9,"Chunk 1000, overlap 10%",0.873766,0.740747,0.512500,0.579270,0.678425,0.799853,0.697427


### change model to GPT-4

In [61]:
highest = results_df.nlargest(3, "Average")
highest

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
28,Parent Retriever 500-100,0.929167,0.899692,0.614198,0.612963,0.884171,0.620998,0.760198
19,"Chunk 3000, overlap 10%",0.895770,0.721733,0.637500,0.800931,0.678137,0.805568,0.756607
32,Ensambler 2,0.886907,0.719502,0.657407,0.589562,0.791583,0.795797,0.740126


In [62]:
result_parent_small_gpt4, results_df = run_and_evaluate(f"Parent Retriever 500-100 -GPT4", parent_document_retriever_small, prompt, llm_gpt4, results_df)
print(result_parent_small_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:07<00:00,  7.11it/s]


The average score is: 0.771540778944933
                           System  Faithfulness  Answer Relevancy  \
0  Parent Retriever 500-100 -GPT4      0.901786            0.8388   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.614198        0.654762           0.886989            0.732711   

    Average  
0  0.771541  


In [63]:
result_3000_gpt4, results_df = run_and_evaluate(f"Chunk 3000, overlap 10%, GPT4", retriever_3000, prompt, llm_gpt4, results_df)
print(result_3000_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.6884865253224509
                          System  Faithfulness  Answer Relevancy  \
0  Chunk 3000, overlap 10%, GPT4       0.90917          0.745605   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0            0.44195        0.560645            0.67732            0.796229   

    Average  
0  0.688487  


In [64]:
result_ensemble2_gpt4, results_df = run_and_evaluate(f"Ensambler 2, GPT-4", ensemble_retriever_2, prompt, llm_gpt4, results_df)
result_ensemble2_gpt4

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

The average score is: 0.7067381361713775


,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,"Ensambler 2, GPT-4",0.822609,0.710117,0.614583,0.630391,0.680792,0.781936,0.706738


In [65]:
results_df.to_csv(f"../ballad/results/results_report.csv", index=False)